# Earnings Call Assistant POC

POC for AI assistant that answers questions about the contents of a company's earnings call.
Uses RAG on chunks.


## Requirements
#### Package Requirements
This notebook was created with the following packages
- python                    3.11
- llama-index               0.12.25
- pandas                    2.2.2
- langchain                 0.3.21

#### Other Requirements
- Environment variable `OPENAI_API_KEY`.  This is needed for LLaMA Index to use its default GPT-3.5 to provide an answer to the query.
- Environment variable `DEEPINFRA_API_KEY`.  This is needed for REST API access LLM models in DeepInfra.

In [1]:
import pandas as pd

## Set up Environment

Setting up environment specific parameters.  Modify these to suit your local environment.

In [2]:

# Locations of the data sources
#

data_root = "../data"         # Directory to the data
ec_dir = "earning_calls"
working_dir = "working"
index_dir = "indices"


In [3]:
import os

# Keys for LLM access
openai_key = os.environ.get("OPENAI_API_KEY")
# hf_key = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
di_key = os.environ.get("DEEPINFRA_API_KEY")

if not openai_key:
    raise EnvironmentError(f"OPENAI_API_KEY must be provided for this notebook to work.  Needed by LLaMA index.")

# if not hf_key:
#     raise EnvironmentError(f"Need HuggingFace token for this notebook to work.  Needed for query extension with DeepSeek-R1"  )

if not di_key:
    raise  EnvironmentError(f"DEEPINFRA_API_KEY is needed to run models in DeepInfra")

In [4]:
#
# Tweak these values
#

# Chunking size
chunk_size = 400
chunk_overlap = 150

# Type of article
article_type = "transcript of the earnings call"
article_name = "MSFT_EC_2Q25"
article_file = "msft/MSFT_FY2Q25__1__m4a_Good_Tape_2025-03-19.txt"

# Query scopes
scope = "Microsoft financial and operational reports"

# LLM models
# llm_model_name = "gpt-4"
# llm_model_name = "gpt-4.5"
llm_model_name = "llama-3"
# llm_model_name = "gemini-2"

# Generation temperature
temperature = 0.6

# Top semantically similar chunks to retrieve for LLM
similarity_top_k = 5

In [5]:
#
# Select an embedding model for vector database.  Here I use LLaMA Index.
#

from llama_index.embeddings.openai import OpenAIEmbedding

# Initialize the OpenAI embedding model
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
# embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Testing
# text = "OpenAI's new embedding models at works"
# print(embed_model.get_text_embedding(text))

In [6]:
# These are steps in this notebook that we want to force refreshing.
# Many of the steps are time-consuming, so I save their results in the data directory.
# If the saved results exists, I will reload them instead of recalculating them.
# Setting any of the steps to True forces the code to recalculate the result for that step.
steps = {
    "chunking": False,                       # Input the article and do chunking
    "query_generation": False,               # Generate possible queries for each chunk
    "indexing": True,                        # Index the chunks with queries
    "qa_indexing": True,                     # Do query-aware indexing
}

def refresh(what:str):
    return what in steps and steps[what]

In [7]:
step_dependencies = {
    "query_generation": ["chunking"],
    "indexing": ["chunking"],
    "qa_indexing": ["query_generation"],
}

more_to_resolve = True
while more_to_resolve:
    more_to_resolve = False
    for step in step_dependencies:
        if not steps[step] and any([steps[s] for s in step_dependencies[step]]):
            steps[step] = True
            more_to_resolve = True

print("Refresh the following steps:")
for s in steps:
    if steps[s]:
        print(f"- {s}")

Refresh the following steps:
- indexing
- qa_indexing


## Reading and Chunking

Read the transcript and chunk it.

In [8]:
from llama_index.core.node_parser import SentenceSplitter

article_path = os.path.join(data_root, ec_dir, article_file)
chunk_path = os.path.join(data_root, working_dir, f"{article_name}_assist_chunks.parquet")

if refresh("chunking") or not os.path.exists(chunk_path):

    # Input
    with open(article_path, "r", encoding="utf-8") as tfd:
        transcript_content = tfd.read()

    # Initialize the SentenceSplitter
    sentence_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # Split the text into chunks
    chunks = sentence_splitter.split_text(transcript_content)

    # Put into Pandas
    chunk_ids = [f"{article_name}_{i:04d}" for i in range(len(chunks))]
    chunk_df = pd.DataFrame(zip(chunk_ids, chunks), columns=["chunk_id", "content"])
    chunk_df = chunk_df.set_index("chunk_id")

    # Save the results
    chunk_df.to_parquet(chunk_path)
else:
    chunk_df = pd.read_parquet(chunk_path)

In [9]:
chunk_df

,content
chunk_id,
MSFT_EC_2Q25_0000,"MSFT_FY2Q25 (1).m4a\n\noperator assistance, pl..."
MSFT_EC_2Q25_0001,They are included as additional clarifying ite...
MSFT_EC_2Q25_0002,Actual results could materially differ because...
MSFT_EC_2Q25_0003,And as AI becomes more efficient and accessibl...
MSFT_EC_2Q25_0004,UBS alone migrated mainframe workloads encompa...
MSFT_EC_2Q25_0005,"Through our strategic partnership, we continue..."
MSFT_EC_2Q25_0006,And GitHub Copilot is increasingly the tool of...
MSFT_EC_2Q25_0007,"Barclays, Carrier Group, Pearson and Universit..."
MSFT_EC_2Q25_0008,"Billions of emails, documents, and chats, hund..."


## Generate Queries for Each Chunk

Generate potential questions for each chunk.  This will help relevancy of vector searches.

In [10]:
import bots

llm = bots.of(llm_model_name)

In [11]:
query_path = os.path.join(data_root, working_dir, f"{article_name}_assist_queries.tsv")

if not os.path.exists(query_path) or refresh("query_generation"):
    query_df = pd.DataFrame(columns=["chunk_id", "question"])
else:
    query_df = pd.read_table(query_path)

query_df

,chunk_id,question
0,MSFT_EC_2Q25_0000,What non-GAAP financial measures will be discu...
1,MSFT_EC_2Q25_0000,Where can the earnings press release and finan...
2,MSFT_EC_2Q25_0000,What is the purpose of providing non-GAAP fina...
3,MSFT_EC_2Q25_0000,How will growth comparisons be made during the...
4,MSFT_EC_2Q25_0000,Where can more detailed outlook slides be found?
...,...,...
425,MSFT_EC_2Q25_0044,Are existing customers increasing their Azure ...
426,MSFT_EC_2Q25_0044,Are new customers making significant Azure com...
427,MSFT_EC_2Q25_0044,What contributed to the broad-based growth in ...
428,MSFT_EC_2Q25_0044,How did M365 Copilot perform in the quarter?


In [12]:
import re
from typing import List
import time

def generate_questions(df: pd.DataFrame, chunk_id: str) -> List[str]:
    """"Generate questions from an article"""

    instruction = """
    The text below is extracted from a {article_type}.
    You are to list possible questions that can be answered by the text.  List only within the scope of {scope}.
    List only the questions.  Do not generate introduction and conclusion.
    ===
    {text}
    """

    max_tries = 5
    text = df.loc[chunk_id]["content"]
    attempts = 0

    while attempts < max_tries:
        try:
            questions = llm.react(
                instruction,
                arguments={
                    "article_type": article_type,
                    "scope": scope,
                    "text": text,
                },
                temperature=temperature
            )["content"]
            return [s for s in re.split(r"\n+", questions) if s]

        except (ValueError, TimeoutError) as e:
            pause = 5 * (attempts + 1)
            print(f"{e}. Retry in {pause} seconds...")
            time.sleep(pause)
            attempts += 1

        time.sleep(2)   # To avoid HuggingFace throttling frequent access

# Testing
# generate_questions(chunk_df, "MSFT_EC_2Q25_0001")

['* What was the revenue growth rate of Microsoft Cloud in the second quarter?',
 '* What was the total revenue of Microsoft Cloud in the second quarter?',
 '* How does Microsoft report growth rates in constant currency?',
 '* What is the year-over-year growth rate of Microsoft Cloud?',
 '* Are the growth rates in constant currency different from the actual growth rates?',
 '* Where can investors find the prepared remarks and transcript of the earnings call?',
 '* What is the trend of enterprise adoption of AI technology?',
 "* What was the previous year's revenue for Microsoft Cloud for comparison?"]

In [13]:
def reformat_question(question: str) -> str | None:
    """ Make question format consistently """
    if "possible customer question" in question:
        # This is a useless introduction by LLM
        return None

    question = re.sub(r"^\s*\d+\.\s*", "", question)  # Remove leading enumeration numbers
    question = re.sub(r"^\s*[-*]{1,2}\s*", "", question)  # Remove "-" or "*" bullet

    return question

# Testing
# print(reformat_question("12. abce da 22"))
# print(reformat_question("  - something something"))
#

In [14]:

from tqdm.notebook import tqdm

if refresh("query_generation") or not os.path.exists(query_path):
    done_chunks = set(query_df["chunk_id"])
    existing_chunks = set(chunk_df.index)
    need_generating = list(existing_chunks - done_chunks)
    need_generating.sort()

    print(f"Generating for {len(need_generating)} articles: {need_generating}")

    with tqdm(need_generating, desc="Generating Questions") as pbar:
        for qid in need_generating:
            pbar.set_postfix_str(qid)
            questions = generate_questions(chunk_df, qid)
            for q in questions:
                q = reformat_question(q)
                if q:
                    query_df.loc[len(query_df)] = [qid, q]

            # Save progress so far
            query_df.to_csv(query_path, sep="\t", index=False)
            pbar.update()

else:
    article_questions = pd.read_table(query_path)


In [15]:
query_df

,chunk_id,question
0,MSFT_EC_2Q25_0000,What non-GAAP financial measures will be discu...
1,MSFT_EC_2Q25_0000,Where can the earnings press release and finan...
2,MSFT_EC_2Q25_0000,What is the purpose of providing non-GAAP fina...
3,MSFT_EC_2Q25_0000,How will growth comparisons be made during the...
4,MSFT_EC_2Q25_0000,Where can more detailed outlook slides be found?
...,...,...
425,MSFT_EC_2Q25_0044,Are existing customers increasing their Azure ...
426,MSFT_EC_2Q25_0044,Are new customers making significant Azure com...
427,MSFT_EC_2Q25_0044,What contributed to the broad-based growth in ...
428,MSFT_EC_2Q25_0044,How did M365 Copilot perform in the quarter?


# Indexing

## Index with Chunk Content Only

In [16]:
from llama_index.core import VectorStoreIndex
#
# Find an answer from LLaMA Index
#

from llama_index.core.evaluation import RetrieverEvaluator
from typing import Tuple

def retrieve(index: VectorStoreIndex, question: str) -> Tuple[str, str]:
    """Search LlamaIndex to find an answer to the question.
    Returns a tuple containing 1) an answer in text, 2) IDs of the candidate articles that may provide information to the answer.
    """
    engine = index.as_query_engine(similarity_top_k=5)
    answer = engine.query(question)
    cids = set([d["chunk_id"] for d in answer.metadata.values()])
    return answer.response, ",".join(cids)


In [17]:

from llama_index.core import Document, VectorStoreIndex, StorageContext, load_index_from_storage

chunk_index_path = os.path.join(data_root, index_dir, f"{article_name}")

if not os.path.exists(chunk_index_path) or refresh("indexing"):
    documents = []

    for idx, row in chunk_df.iterrows():
        cid = idx
        text = row["content"]
        documents.append(
            Document(text=text, metadata={"chunk_id": cid}, doc_id=str(idx))
        )

    # Create a vector index with the specified embedding model
    chunk_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

    # Save the index for later use
    chunk_index.storage_context.persist(persist_dir=chunk_index_path)

else:
    storage_context = StorageContext.from_defaults(persist_dir=chunk_index_path)
    chunk_index = load_index_from_storage(storage_context)


In [18]:
# Testing
print(retrieve(chunk_index, "What is Microsoft total FY2Q25營收 and 季增, 年增?"))
print(retrieve(chunk_index, "FY2Q25 Azure revenue"))
print(retrieve(chunk_index, "Fill in values in the blanks: Microsoft total FY2Q25營業利益 $___，季增___%/年增___%，優於優於財測中值/市場預期___%/___%"))

("Microsoft's total FY2Q25 revenue is $40 billion, representing a 21% year-over-year increase.", 'MSFT_EC_2Q25_0018,MSFT_EC_2Q25_0023,MSFT_EC_2Q25_0001,MSFT_EC_2Q25_0020,MSFT_EC_2Q25_0022')
('Between $25.9 to $26.2 billion.', 'MSFT_EC_2Q25_0015,MSFT_EC_2Q25_0012,MSFT_EC_2Q25_0019,MSFT_EC_2Q25_0014,MSFT_EC_2Q25_0020')
('Microsoft total FY2Q25 operating income $20.6 billion, quarter increase 32%/year increase 30%, better than consensus/market expectations 5%/3%', 'MSFT_EC_2Q25_0018,MSFT_EC_2Q25_0017,MSFT_EC_2Q25_0021,MSFT_EC_2Q25_0023,MSFT_EC_2Q25_0022')


## Index with Question-Context Pairs (Query-Aware Indexing)

In [19]:

from llama_index.core import Document, VectorStoreIndex, StorageContext, load_index_from_storage

qa_index_path = os.path.join(data_root, index_dir, f"{article_name}_qa")

if not os.path.exists(qa_index_path) or refresh("qa_indexing"):
    documents = []

    for idx, row in query_df.iterrows():
        cid = row["chunk_id"]
        question = row["question"]
        text = chunk_df.loc[cid]["content"]
        documents.append(
            Document(text=f"{question}\n====\n{text}", metadata={"chunk_id": cid}, doc_id=str(idx))
        )

    # Create a vector index with the specified embedding model
    query_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

    # Save the index for later use
    query_index.storage_context.persist(persist_dir=qa_index_path)

else:
    storage_context = StorageContext.from_defaults(persist_dir=qa_index_path)
    query_index = load_index_from_storage(storage_context)


In [20]:
# Testing
print(retrieve(query_index, "What is Microsoft total FY2Q25營收 and 季增, 年增?"))
print(retrieve(query_index, "FY2Q25 Azure revenue"))
print(retrieve(query_index, "Fill in values in the blanks: Microsoft total FY2Q25營業利益 $___，季增___%/年增___%，優於優於財測中值/市場預期___%/___%"))

("Microsoft's total revenue in the second quarter was over $40 billion, representing a 21% year-over-year increase.", 'MSFT_EC_2Q25_0018,MSFT_EC_2Q25_0023,MSFT_EC_2Q25_0001')
('Between 31% and 32% in constant currency.', 'MSFT_EC_2Q25_0015,MSFT_EC_2Q25_0020,MSFT_EC_2Q25_0014')
('Microsoft total FY2Q25 operating income $16.5 billion, quarter growth 15%/year growth 18%, better than consensus/market expectations 2%/3%.', 'MSFT_EC_2Q25_0023')
